In [62]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import pytz

In [65]:
# Read in Transaction Data
data = pd.read_csv('transaction.csv')
data.head(5)

,value,block_timestamp
0,0,1668124739
1,117286251058621419,1668124739
2,0,1668124739
3,0,1668124739
4,0,1668124739


In [66]:
"""
def index_calc(data):

            :param data: dataframe containing transaction value and timestamp of the transaction (block_timestamp)
            :type data: pandas.DataFrame
            :param start: the start date of calculation 00:00:00 UTC
            :type start: datetime.date object
            :param end: the end date of calculation 23:59:59 UTC
            :type end: datetime.date object

    returns pandas dataframe with decentralization index value and date
"""

def index_calc(data, start, end):
    tz = pytz.timezone('UTC')
    data['datetime'] = data['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x,tz))
    duration= pd.date_range(start=start, end=end)
    days = np.size(duration)
    Entropy = pd.DataFrame(np.zeros(days), columns=['val'])
    Entropy['date'] = duration

    for i in range(0, days):
        start_date = start + timedelta(days=i)
        end_date = start_date + timedelta(days=1)
        Entropy.loc[i,'val'] = index(data[(data['datetime'].dt.date >= start_date) & (data['datetime'].dt.date < end_date)].copy())
    
    return Entropy

def index(df):
    df['value'] = df['value'].astype(float)
    df['prop'] = df['value']/df['value'].sum() 
    H = -df['prop']*np.log2(df['prop'])
    E = H.sum()
    V = 2**E
    return V

In [82]:
#input transaction data and desired date range
#returns index value 1 if specification range exceeds data 
start = date(2022,11,10)
end = date(2022,11,13)
results = index_calc(data,start,end)

/Users/xma/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xma/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [83]:
#sample data from 2022-11-10 to 2022-11-11
results

,val,date
0,13.089093,2022-11-10
1,13.558045,2022-11-11
2,1.000000,2022-11-12
3,1.000000,2022-11-13
